The cohesion function is in bnp-anomaly/bnpy/allocmodel/mix/DPMixtureModel.py

In [ ]:
def calcLocalParams(Data, LP, G=0, Elogbeta=None, nnzPerRowLP=None, **kwargs):
    """ Compute local parameters for each data item.

    Parameters
    -------
    Data : bnpy.data.DataObj subclass

    LP : dict
        Local parameters as key-value string/array pairs
        * E_log_soft_ev : 2D array, N x K
            E_log_soft_ev[n,k] = log p(data obs n | comp k)

    Returns
    -------
    LP : dict
        Local parameters, with updated fields
        * resp : 2D array, size N x K array
            Posterior responsibility each comp has for each item
            resp[n, k] = p(z[n] = k | x[n])
    """
    lpr = LP['E_log_soft_ev']
    lpr += Elogbeta
    K = LP['E_log_soft_ev'].shape[1]
    if nnzPerRowLP and (nnzPerRowLP > 0 and nnzPerRowLP < K):
        # SPARSE Assignments
        LP['spR'] = sparsifyLogResp(lpr, nnzPerRow=nnzPerRowLP)
        assert np.all(np.isfinite(LP['spR'].data))
        LP['nnzPerRow'] = nnzPerRowLP
    else:
        # DENSE Assignments
        # Calculate exp in numerically stable manner (first subtract the max)
        #  perform this in-place so no new allocations occur
        NumericUtil.inplaceExpAndNormalizeRows(lpr)
        ### Changes here for cohesion function
        if G > 0:
            posterior_nu = kwargs["Post"].nu  # shape K x D array
            posterior_beta = kwargs["Post"].beta  # shape K x D array
            g_func = cohesion(Data.X, lpr, posterior_nu, posterior_beta, G)
            lpr = np.log(lpr) + g_func
            NumericUtil.inplaceExpAndNormalizeRows(lpr)
        LP['resp'] = lpr
    return LP


def cohesion(in_arr, assgn_prob, nu, beta, G):
    # Cohession Mark I
    if G == 1:
        num_k = int(nu.size)
        data_arr = in_arr.repeat(num_k, 1)

        assgn = np.argmax(assgn_prob, axis=1)
        one_hot_y = np.eye(num_k, dtype=np.int)
        one_hot_y = one_hot_y[assgn]

        g_calc = np.ones_like(assgn_prob) / num_k
        for idx in range(num_k):
            num_data = np.sum(one_hot_y[:,idx])
            if num_data > 0:
                D_tk = data_arr[one_hot_y[:,idx], idx]
                dmean = np.mean(D_tk)
                dstd = np.std(D_tk) + 1e-8
                g_calc[:,idx] = np.cumsum(norm.logpdf(in_arr, dmean, dstd))
    #Cohession Mark II
    elif G == 2:
        num_k = int(nu.size)
        data_arr = in_arr.repeat(num_k, 1)

        assgn = np.argmax(assgn_prob, axis=1)
        one_hot_y = np.eye(num_k, dtype=np.int)
        one_hot_y = one_hot_y[assgn]

        g_calc = np.ones_like(assgn_prob) / num_k
        for idx in range(num_k):
            num_data = np.sum(one_hot_y[:,idx])
            if num_data > 0:
                D_tk = data_arr[one_hot_y[:,idx], idx]
                dmean = np.mean(D_tk)
                dstd = np.std(D_tk) + 1e-8
                df = num_data -1 if num_data > 1 else 1
                g_calc[:,idx] = np.cumsum(student_t.logpdf(in_arr, df, dmean, dstd))
    # No Cohesion
    else:
        g_calc = 0
    return g_calc